In [1]:
import sympy as sp
from ham_to_sparse_matrix import *
from constants import *
import scipy.sparse.linalg

In [3]:
bosonI=sp.expand( V(qn)*(qnP1-qnM1)/2 )

In [4]:
bosonI

V(q_{n})*q_{n+1}/2 - V(q_{n})*q_{n-1}/2

In [11]:
N=3
aVal=1

# depends on finite-difference method
qs=[SiteSymbol('q',str(i)) for i in range(-1,N+1,1)]
# maybe make qs a normal site list
# and make an extra boundaryQs list for q[-1], q[N]
# counting would be normal computer science way for rest of code.

ps=[SiteSymbol('p',str(i)) for i in range(-1,N+1,1)] # don't really need extras
aops=[SiteSymbol('a',str(i)) for i in range(-1,N+1,1)]
adags=[SiteSymbol('a^{\dagger}',str(i)) for i in range(-1,N+1,1)]
xs=[SiteSymbol('\chi',str(i)) for i in range(-1,N+1,1)]
xdags=[SiteSymbol('\chi^{\dagger}',str(i)) for i in range(-1,N+1,1)]

# note this is exactly hardcoded for this finite difference method.
bcType = 'periodic'
boundaryConditions = {}
if bcType == 'periodic':
    boundaryConditions = {qs[0]: qs[N], qs[N+1]: qs[1],
                          xs[0]: -xs[N], xs[N+1]: -xs[1], 
                          xdags[0]: -xdags[N],  xdags[N+1]: -xdags[1]
                         }
    
elif bcType == 'dirichlet':
    boundaryConditions = { qs[0]: 0, qs[N+1]: 0,
                          xs[0]: 0, xs[N+1]: 0,
                          xdags[0]: 0, xdags[N+1]:0
                         }

ham=0
    
for i in range(1,N+1):
    ham+=(bosonI).subs({
        pn: ps[i],
        qn: qs[i], qnP1: qs[i+1], qnM1: qs[i-1],
        xn: xs[i], xnP1: xs[i+1],
        xdn: xdags[i], xdnP1: xdags[i+1]
    }).subs(boundaryConditions)
    
def potential(n):
    # m*q with m=1
    return qs[n]

potentialSubs={}
for n in range(1,N+1):
    potentialSubs[V(qs[n])]=potential(n)


ham=sp.simplify(ham.subs(potentialSubs))

In [12]:
ham

(q_{0}*q_{1} - q_{0}*q_{2} - q_{1}*q_{0} + q_{1}*q_{2} + q_{2}*q_{0} - q_{2}*q_{1})/2

In [13]:
m=1

HOdofSubs = {}
#offset because of BC
for i in range(1,N+1):
    HOdofSubs[qs[i]] = 0.5*sp.sqrt(2/m)*(aops[i] + adags[i])
    HOdofSubs[ps[i]] = complex(0,1)*sp.sqrt(2*m)*(adags[i] - aops[i])/2 

hoHam=sp.expand(ham.subs(HOdofSubs))
hoHam=sp.nsimplify(hoHam,tolerance=1e-8)
hoHam

a^{\dagger}_{0}*a^{\dagger}_{1}/4 - a^{\dagger}_{0}*a^{\dagger}_{2}/4 + a^{\dagger}_{0}*a_{1}/4 - a^{\dagger}_{0}*a_{2}/4 - a^{\dagger}_{1}*a^{\dagger}_{0}/4 + a^{\dagger}_{1}*a^{\dagger}_{2}/4 - a^{\dagger}_{1}*a_{0}/4 + a^{\dagger}_{1}*a_{2}/4 + a^{\dagger}_{2}*a^{\dagger}_{0}/4 - a^{\dagger}_{2}*a^{\dagger}_{1}/4 + a^{\dagger}_{2}*a_{0}/4 - a^{\dagger}_{2}*a_{1}/4 + a_{0}*a^{\dagger}_{1}/4 - a_{0}*a^{\dagger}_{2}/4 + a_{0}*a_{1}/4 - a_{0}*a_{2}/4 - a_{1}*a^{\dagger}_{0}/4 + a_{1}*a^{\dagger}_{2}/4 - a_{1}*a_{0}/4 + a_{1}*a_{2}/4 + a_{2}*a^{\dagger}_{0}/4 - a_{2}*a^{\dagger}_{1}/4 + a_{2}*a_{0}/4 - a_{2}*a_{1}/4

In [16]:
cutoff=6
hamMat=convert_to_matrix(hoHam,cutoff,N,aops,adags,xs,xdags)

In [17]:
hamMat

<1728x1728 sparse matrix of type '<class 'numpy.complex64'>'
	with 0 stored elements in Compressed Sparse Row format>

as expected in "Supersymmetry on the lattice" by rittenberg & yankielowicz"